# Search Engine

In [1]:
import nltk
nltk.download('stopwords');
import time
from collections import defaultdict
from array import array
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import math
import numpy as np
import collections
import pandas as pd
import json
from numpy import linalg as la
import ast
#Import the necessary methods from tweepy library
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
from tweepy import API
from tweepy import Cursor

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Wenjie\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
## access token informations 
access_token1 = "1240651902712516613-BF3CepSxPUoMr3lDkptNt08f2GwaGZ"
access_token_secret1 = "79ga9EksEgtgpHGJhXy7A4ujb31pw1AjwP8lOlaYtmDdk"

consumer_key1 = "ieusj8Wof83kmsVe9PDLJQqn3"
consumer_secret1 = "EpZYsMfEqvHSwFQyAfz2wCdVujPTtr7JQtvkBt1g9sMg5t0m3B"

#Generate API
auth = OAuthHandler(consumer_key1, consumer_secret1)
auth.set_access_token(access_token1, access_token_secret1)
api = API(auth_handler=auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

###### Load data from json

In [4]:
#load data from output.json scrapped from twitter
with open("../other-outputs/output.json", "rb") as f:
    data = f.readlines()
    data = [json.loads(str_) for str_ in data]

df_tweets = pd.DataFrame.from_records(data)

# Get unique tweets
tweets=df_tweets[df_tweets["retweeted_status"].isna()]
df_tweets.head()

,created_at,id,id_str,text,source,truncated,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,...,lang,timestamp_ms,quoted_status_id,quoted_status_id_str,quoted_status,quoted_status_permalink,possibly_sensitive,extended_tweet,display_text_range,extended_entities
0,Sat Dec 05 10:11:06 +0000 2020,1335164788498980864,1335164788498980864,RT @JoeDanMedia: In 2009... Santelli lit the T...,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,None,NaN,None,...,en,1607163066397,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Sat Dec 05 10:11:06 +0000 2020,1335164788767277061,1335164788767277061,RT @CNNPolitics: President-elect Joe Biden on ...,"<a href=""http://twitter.com/#!/download/ipad"" ...",False,NaN,None,NaN,None,...,en,1607163066461,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Sat Dec 05 10:11:06 +0000 2020,1335164789060984832,1335164789060984832,Arbroath #arbroath #covid,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,None,NaN,None,...,en,1607163066531,1.335162e+18,1335162235216420864,{'created_at': 'Sat Dec 05 10:00:57 +0000 2020...,"{'url': 'https://t.co/HrzZRGUtCS', 'expanded':...",NaN,NaN,NaN,NaN
3,Sat Dec 05 10:11:06 +0000 2020,1335164790180864001,1335164790180864001,RT @johnnymorls: I think that wraps things up ...,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,None,NaN,None,...,en,1607163066798,1.335132e+18,1335132252800503808,{'created_at': 'Sat Dec 05 08:01:49 +0000 2020...,"{'url': 'https://t.co/738ZoPEBv7', 'expanded':...",NaN,NaN,NaN,NaN
4,Sat Dec 05 10:11:06 +0000 2020,1335164790365368320,1335164790365368320,RT @CAOYI170610: According to a recent US CDC ...,"<a href=""https://mobile.twitter.com"" rel=""nofo...",False,NaN,None,NaN,None,...,en,1607163066842,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


##### Update (Don't execute this blog)

In [4]:
def update(tweets):
    """
    Update number of the tweet's likes and retweed.
    """
    for index, tweet in tweets.iterrows():
        try:
            tweet = api.get_status(tweet["id"])
            tweets.loc[index, "favorite_count"] = tweet.favorite_count
            tweets.loc[index, "retweet_count"] = tweet.retweet_count
        except:
            pass


In [5]:
update(tweets)


NameError: name 'tweets' is not defined

In [ ]:
tweets["favorite_count"].head()

In [ ]:
tweets.to_csv('../other-outputs/tweets.tsv', sep = '\t', index=False)

#####  Load data from TSV

In [7]:
tweets = pd.read_csv("../other-outputs/tweets.tsv", sep='\t', index_col=0)


###### Consult the more likes and more retweets.

In [5]:
more_likes = 0
text_with_more_likes = ""
for index, tweet in tweets.iterrows():
    likes_tweet = tweet["favorite_count"]
    if likes_tweet > more_likes:
        more_likes = likes_tweet
        text_with_more_likes =  tweet["text"]
print("Most liked tweet: {}, with a total of {} likes.".format(text_with_more_likes, more_likes))

Most liked tweet: , with a total of 0 likes.


In [6]:
more_retweets = 0
text_with_more_retweets = ""
for index, tweet in tweets.iterrows():
    num_retweets = tweet["retweet_count"]
    if num_retweets > more_retweets:
        more_retweets = num_retweets
        text_with_more_retweets = tweet["text"]
        
print("Most retweets tweet: {}, with a total of {} retweets.".format(text_with_more_retweets, more_retweets))

Most retweets tweet: , with a total of 0 retweets.


##### Pre-processing the Tweets

In [10]:
def getTerms(tweet):
    """
    Preprocess the article text (title + body) removing stop words, stemming,
    transforming in lowercase and return the tokens of the text.
    
    Argument:
    tweet -- string (text) to be preprocessed
    
    Returns:
    tweet - a list of tokens corresponding to the input text after the preprocessing
    """
        
    stemming = PorterStemmer()
    stops = set(stopwords.words("english"))
    tweet =  tweet.lower() ## Transform in lowercase
    tweet = tweet.replace("#", "") ## removing the “#” from the word 
    tweet =  tweet.split(" ") ## Tokenize the text to get a list of terms
    tweet = [word for word in tweet if word not in stops] #Remove stopwords. 
    tweet =[stemming.stem(word) for word in tweet] 
    return tweet

##### Inverted-index 

In [11]:
def create_index_tfidf(tweets, numTweets):
    """
    Implement the inverted index and compute tf, df and idf
    
    Argument:
    lines -- collection of Wikipedia articles
    numTweets -- total number of documents
    
    Returns:
    index - the inverted index (implemented through a python dictionary) containing terms as keys and the corresponding 
    list of document these keys appears in (and the positions) as values.
    tf - normalized term frequency for each term in each document
    df - number of documents each term appear in
    idf - inverse document frequency of each term
    """
    index=defaultdict(list) 
    tf=defaultdict(list) #term frequencies of terms in documents (documents in the same order as in the main index)
    df=defaultdict(int)         #document frequencies of terms in the corpus
    IdIndex = {} # dictionary to map page titles to page ids
    idf=defaultdict(float) # dictionary with inversed document frequency 
    
    for i,tweet in tweets.iterrows():
        tweet_id = i
        terms = getTerms(tweet["text"]) #page_title + page_text
        userid = int(tweet["id"])            
        IdIndex[tweet_id]=userid  ## we do not need to apply get terms to title because it used only to print titles and not in the index
        ## ===============================================================        
        ## create the index for the current doc and store it in termdictPage
        ## termdictPage ==> { ‘term1’: [currentdoc, [list of positions]], ...,‘termn’: [currentdoc, [list of positions]]}
        
        ## Example: if the curr_doc has id 1 and his text is 
        ## "web retrieval information retrieval":
        
        ## termdictPage ==> { ‘web’: [1, [0]], ‘retrieval’: [1, [1,4]], ‘information’: [1, [2]]}
        
        ## the term ‘web’ appears in document 1 in positions 0, 
        ## the term ‘retrieval’ appears in document 1 in positions 1 and 4
        ## ===============================================================
        
        termdictPage={}

        for position, term in enumerate(terms): # terms contains page_title + page_text. Loop over all terms
            try:
                # if the term is already in the index for the current page (termdictPage)
                # append the position to the corrisponding list
                
                termdictPage[term][1].append(position)  
            except:
                # Add the new term as dict key and initialize the array of positions and add the position
                termdictPage[term]=[tweet_id, array('I',[position])] #'I' indicates unsigned int (int in python)
        
        #normalize term frequencies
        # Compute the denominator to normalize term frequencies
        # norm is the same for all terms of a document.
        norm=0
        for term, posting in termdictPage.items(): 
            # posting is a list containing doc_id and the list of positions for current term in current document: 
            # posting ==> [currentdoc, [list of positions]] 
            # you can use it to inferr the frequency of current term.
            norm+=len(posting[1])**2
        norm=math.sqrt(norm)
        
        #calculate the tf(dividing the term frequency by the above computed norm) and df weights
        for term, posting in termdictPage.items():     
            # append the tf for current term (tf = term frequency in current doc/norm)
            tf[term].append(np.round(len(posting[1])/norm,4))  
            #increment the document frequency of current term (number of documents containing the current term)
            df[term]+= 1  # increment df for current term   
            
        #merge the current page index with the main index
        for termpage, postingpage in termdictPage.items():
            index[termpage].append(postingpage)
        
         # Compute idf
        for term in df:
            idf[term] = np.round(np.log(float(numTweets/df[term])),4)                   
                    
    return index, tf, df, idf, IdIndex

In [12]:
#Generate Index and idIndex. 
start_time = time.time()
numTweets = len(tweets)
index, tf, df, idf, idIndex = create_index_tfidf(tweets, numTweets)
print("Total time to create the index: {} seconds" .format(np.round(time.time() - start_time,2)))


Total time to create the index: 39.57 seconds


##### Ranking Score

In [13]:
def rankDocuments(terms, docs, index, idf, tf, idIndex):
    """
    Perform the ranking of the results of a search based on the tf-idf weights
    
    Argument:
    terms -- list of query terms
    docs -- list of documents, to rank, matching the query
    index -- inverted index data structure
    idf -- inverted document frequencies
    tf -- term frequencies
    idIndex -- mapping between page id and tweet id
    
    Returns:
    Print the list of ranked documents
    """
    docVectors=defaultdict(lambda: [0]*len(terms)) 
    queryVector=[0]*len(terms)    
                                                                                                                                                                                                                                                                     
    # compute the norm for the query tf
    query_terms_count = collections.Counter(terms) # get the frequency of each term in the query. 
    # Example: collections.Counter(["hello","hello","world"]) --> Counter({'hello': 2, 'world': 1})    
    query_norm = la.norm(list(query_terms_count.values()))
    
    
    for termIndex, term in enumerate(terms): #termIndex is the index of the term in the query
        if term not in index:
            continue
                    
        ## Compute tf*idf(normalize tf as done with documents)
        queryVector[termIndex]=query_terms_count[term]/query_norm * idf[term]

        # Generate docVectors for matching docs
        for docIndex, (doc, postings) in enumerate(index[term]):
            # Example of [docIndex, (doc, postings)]
            # 0 (26, array('I', [1, 4, 12, 15, 22, 28, 32, 43, 51, 68, 333, 337]))
            # 1 (33, array('I', [26, 33, 57, 71, 87, 104, 109]))
            # term is in doc 26 in positions 1,4, .....
            # term is in doc 33 in positions 26,33, .....
            
            if doc in docs:
                tweetInfo = tweets[tweets["id"] == idIndex[doc]]
                like =  tweetInfo["favorite_count"].values
                retweet = tweetInfo["retweet_count"].values
                docVectors[doc][termIndex]=tf[term][docIndex] * idf[term]

    # calculate the score of each doc
    # compute the cosine similarity between queyVector and each docVector:    
    docScores=[ [np.dot(curDocVec, queryVector), doc] for doc, curDocVec in docVectors.items() ]
    docScores.sort(reverse=True)
    resultDocs=[x[1] for x in docScores]
    #print document titles instead if document id's
    #resultDocs=[ idIndex[x] for x in resultDocs ]
    if len(resultDocs) == 0:
        print("No results found, try again")
        query = input()
        docs = search_tf_idf(query, index)    
    #print ('\n'.join(resultDocs), '\n')
    return resultDocs

##### My score

In [14]:
def rankDocumentsWithLikes(terms, docs, index, idf, tf, idIndex):
    """
    In this function we compute the out ranking score with like and retweets counts. 
    Because in this case, the amount of rwteets and the number of user's likes
    are also one of the important factors for ranking. 
    
    Argument:
    terms -- list of query terms
    docs -- list of documents, to rank, matching the query
    index -- inverted index data structure
    idf -- inverted document frequencies
    tf -- term frequencies
    idIndex -- mapping between page id and tweet id
    
    Returns:
    Print the list of ranked documents
    """
    docVectors=defaultdict(lambda: [0]*len(terms)) 
    queryVector=[0]*len(terms)    
                                                                                                                                                                                                                                                                     
    # compute the norm for the query tf
    query_terms_count = collections.Counter(terms) # get the frequency of each term in the query. 
    # Example: collections.Counter(["hello","hello","world"]) --> Counter({'hello': 2, 'world': 1})    
    query_norm = la.norm(list(query_terms_count.values()))
    
    
    for termIndex, term in enumerate(terms): #termIndex is the index of the term in the query
        if term not in index:
            continue
                    
        ## Compute tf*idf(normalize tf as done with documents)
        queryVector[termIndex]=query_terms_count[term]/query_norm * idf[term]

        # Generate docVectors for matching docs
        for docIndex, (doc, postings) in enumerate(index[term]):
            # Example of [docIndex, (doc, postings)]
            # 0 (26, array('I', [1, 4, 12, 15, 22, 28, 32, 43, 51, 68, 333, 337]))
            # 1 (33, array('I', [26, 33, 57, 71, 87, 104, 109]))
            # term is in doc 26 in positions 1,4, .....
            # term is in doc 33 in positions 26,33, .....
            
            if doc in docs:
                tweetInfo = tweets[tweets["id"] == idIndex[doc]]
                like =  tweetInfo["favorite_count"].values
                retweet = tweetInfo["retweet_count"].values
                docVectors[doc][termIndex]=tf[term][docIndex] * idf[term] + (like + retweet)

    # calculate the score of each doc
    # compute the cosine similarity between queyVector and each docVector:    
    docScores=[ [np.dot(curDocVec, queryVector), doc] for doc, curDocVec in docVectors.items() ]
    docScores.sort(reverse=True)
    resultDocs=[x[1] for x in docScores]
    #print document titles instead if document id's
    #resultDocs=[ idIndex[x] for x in resultDocs ]
    if len(resultDocs) == 0:
        print("No results found, try again")
        query = input()
        docs = search_tf_idf(query, index)    
    #print ('\n'.join(resultDocs), '\n')
    return resultDocs

##### Search with tf-idf ranking

In [15]:
def search_tf_idf(query, index):
    '''
    output is the list of documents that contain any of the query terms. 
    So, we will get the list of documents for each query term, and take the union of them.
    '''
    query=getTerms(query)
    docs=set()
    for term in query:
        try:
            # store in termDocs the ids of the docs that contain "term"                        
            termDocs=[posting[0] for posting in index[term]]
            
            # docs = docs Union termDocs
            docs |= set(termDocs)
        except:
            #term is not in index
            pass
    docs=list(docs)
    ranked_docs = rankDocuments(query, docs, index, idf, tf, idIndex)   
    return ranked_docs

In [16]:
print("Insert your query:\n")
query = input()
ranked_docs = search_tf_idf(query, index)    
top = 20



Insert your query:

covid


###### My-Score + cosine similarity

In [18]:
def search_tf_idf(query, index):
    '''
    output is the list of documents that contain any of the query terms. 
    So, we will get the list of documents for each query term, and take the union of them.
    '''
    query=getTerms(query)
    docs=set()
    for term in query:
        try:
            # store in termDocs the ids of the docs that contain "term"                        
            termDocs=[posting[0] for posting in index[term]]
            
            # docs = docs Union termDocs
            docs |= set(termDocs)
        except:
            #term is not in index
            pass
    docs=list(docs)
    ranked_docs = rankDocumentsWithLikes(query, docs, index, idf, tf, idIndex)   
    return ranked_docs

In [19]:
print("Insert your query:\n")
query = input()
ranked_docs = search_tf_idf(query, index)    
top = 20

Insert your query:

covid


##### Output

In [20]:
print("\n======================\nTop {} results out of {} for the seached query:\n".format(top, len(ranked_docs)))
resultTmp = []

for d_id in ranked_docs[:top] :
    ID = idIndex[d_id]
    #print("page_id= {} - page_title: {}".format(d_id, titleIndex[d_id]))
    tweetTmp = tweets[tweets["id"] == ID]
    tweet = tweetTmp["text"].values[0]
    username = ast.literal_eval(tweetTmp["user"].values[0])["screen_name"]
    date = tweetTmp["created_at"].values[0]
    hashtags_lists = ast.literal_eval(tweetTmp["entities"].values[0])["hashtags"]
    hashtags = [tag["text"] for tag in hashtags_lists]
    likes = tweetTmp["favorite_count"].values[0]
    retweets = tweetTmp["retweet_count"].values[0]
    url = ast.literal_eval(tweetTmp["user"].values[0])["url"]
    
    resultTmp.append([tweet,username,date,hashtags,likes,retweets,url])
    
result = pd.DataFrame(resultTmp,columns=["Tweet", "Username", "Date", "Hashtags", "Likes", "Retweets", "Url"])
result


Top 20 results out of 317 for the seached query:



,Tweet,Username,Date,Hashtags,Likes,Retweets,Url
0,"Pelosi bullish on COVID relief: ""We cannot lea...",thehill,Sat Dec 05 10:21:03 +0000 2020,[],105,23,http://www.thehill.com
1,This is a minor example of the kind of short-c...,smolrobots,Sat Dec 05 12:29:54 +0000 2020,[],79,7,https://www.patreon.com/thomasheasmanhunt
2,- Vaccine in care homes in the next two weeks\...,BBCNews,Sat Dec 05 10:12:53 +0000 2020,[],63,20,http://www.bbc.co.uk/news
3,🙄😏the new normal~\nme: yayyyy tested covid neg...,zainaconda,Sat Dec 05 10:18:33 +0000 2020,[],58,3,None
4,"Plea In Supreme Court Over ""Exorbitant Money"" ...",ndtv,Sat Dec 05 10:21:09 +0000 2020,[],43,5,http://www.ndtv.com/
5,One of my student work colleagues had to take ...,TheMindsculpter,Sat Dec 05 10:20:14 +0000 2020,[],26,2,None
6,"To be fair to fat Joe, he's only done what thi...",ScouseMan_,Sat Dec 05 10:12:34 +0000 2020,[],26,0,None
7,Teenager fined £120 after telling police there...,DailyMailUK,Sat Dec 05 10:20:06 +0000 2020,[],20,5,http://dailymail.co.uk/news
8,My covid self isolation is over and I can leav...,SpeakSarahSpeak,Sat Dec 05 10:20:19 +0000 2020,[],20,0,http://www.RippleEnergy.com
9,A massive well done and thank you to our #Volu...,CraigAHarman,Sat Dec 05 10:20:42 +0000 2020,[Volunteers],14,2,http://www.sja.org.uk


### Word2vec

In [ ]:
def get_tokens(docs, preprocess=preprocess_string, verbose=10000):
    
    for i, doc in enumerate(docs):
        yield preprocess(doc)#  preprocess
        
        # print progress if needed
        if verbose > 0 and (i + 1) % verbose == 0:
            print(f"Progress: {i + 1}")
    return docs